In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from useful_scit.imps import * 
import useful_scit.util.zarray as za

In [3]:
import wrf_management.modules.CompressOut as CO
import sqlite3

In [45]:
# path  = '/Volumes/mbProD/Downloads/wrf_test_d01/'
pc = 'taito'
pc = 'mac'

if pc is 'taito':
    path = '/proj/atm/saltena/runs/run_2019_05_15/wrf'
    path_out  = '/proj/atm/saltena/runs/run_2019_05_15/wrf_compressed'
    db_path = os.path.join(path_out,'zip.sqlite')
    patt = '2017-12-0*'
    last_date_lock = True
    
if pc is 'mac':
    path = '/Volumes/mbProD/Downloads/wrf_test_d01/'
    path_out  = '/private/tmp/co_out/'
    db_path = os.path.join(path_out,'zip.sqlite')
    patt = 'd0*'
    last_date_lock = False
self = CO.Compresser(path,
                     path_out,
                     db_path,
                     pattern=patt, 
                     lock_last_date = last_date_lock,
                     source_path_is_file=False
                    )

2019-06-19 16:33:53,657 - DEBUG - number of files is 80
2019-06-19 16:33:53,667 - DEBUG - number of file remaining after dropping symlinks is 20
2019-06-19 16:33:56,563 - DEBUG - number of file remaining after dropping not cdf files is 20
2019-06-19 16:33:59,857 - DEBUG - number of files with compress level lower than target is 0
2019-06-19 16:33:59,857 - CRITICAL - all files already compressed or no files matched pattern. exiting


SystemExit: 1

/Users/diego/miniconda3/envs/b36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
row = self.compress_out_df.iloc[0]

In [9]:
self.get_and_zip_next_row(move=True)

2019-06-19 02:03:32,805 - DEBUG - compressing:wrfout_d01_2017-12-25_04:00:00
2019-06-19 02:03:47,217 - DEBUG - {'moving /private/tmp/co_out/wrfout_d01_2017-12-25_04:00:00 to /Volumes/mbProD/Downloads/wrf_test_d01/wrfout_d01_2017-12-25_04:00:00'}
2019-06-19 02:03:49,483 - DEBUG - double check fine. deleting /private/tmp/co_out/wrfout_d01_2017-12-25_04:00:00_source_back_up


In [100]:
self.merge_update_dfs()

In [101]:

query = f'''
    select {CO.DATE_COL} from {self.files_table_name}
    order by {CO.DATE_COL} desc limit 1
    '''
with sqlite3.connect(self.db_path) as con:
    last_date = pd.read_sql_query(query,con).iloc[0][CO.DATE_COL]
    

In [102]:
last_date

1514228400000000000

In [97]:
query = f'''
    delete from {self.files_table_name} where {CO.DATE_COL}=={last_date}
    '''
with sqlite3.connect(self.db_path) as con:
    cursor = con.cursor()
    cursor.execute(query)
    con.commit()


In [8]:
while True:
    self.get_and_zip_next_row()

2019-06-18 19:50:45,963 - DEBUG - compressing:wrfout_d01_2017-12-25_07:00:00
2019-06-18 19:51:05,022 - DEBUG - compressing:wrfout_d01_2017-12-25_09:00:00
2019-06-18 19:51:23,442 - DEBUG - compressing:wrfout_d01_2017-12-25_11:00:00
2019-06-18 19:51:41,827 - DEBUG - compressing:wrfout_d01_2017-12-25_13:00:00
2019-06-18 19:52:00,474 - DEBUG - compressing:wrfout_d01_2017-12-25_15:00:00
2019-06-18 19:52:19,262 - DEBUG - compressing:wrfout_d01_2017-12-25_17:00:00
2019-06-18 19:52:37,954 - DEBUG - compressing:wrfout_d01_2017-12-25_19:00:00
2019-06-18 19:52:47,720 - CRITICAL - we are done! no more files to compress


SystemExit: 0

/Users/diego/miniconda3/envs/b36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
sadf

NameError: name 'sadf' is not defined

In [15]:
CO.run_srun('hola',time_minutes=20,memory=4000,parallel_type='serial',n_cpus=1)

2019-06-19 12:13:02,301 - DEBUG - ['srun', '-t', '20', '-p', 'serial', '-c', '1', '--mem', '4000', 'hola']


FileNotFoundError: [Errno 2] No such file or directory: 'srun': 'srun'

In [35]:
path = '/Volumes/mbProD/Downloads/wrf_test_d01/wrfout_d01_2017-12-25_10:00:00'
# path = '/private/tmp/co_out/wrfout_d01_2017-12-25_18:00:00'
ds = xr.open_dataset(path)
variables = list(ds.variables)
compression_list = []
for var in variables:
    c = ds[var].encoding.get('complevel', 0)
    compression_list.append(c)
compression_list = np.array(compression_list)

In [36]:
np.mean(compression_list)

0.0